In [1]:
import tensorflow as tf 

In [ ]:
hello_constant = tf.constant("Hello_World")
A = tf.constant(1234)
B = tf.constant([ [123,456,789] ])
C = tf.constant([ [123,456,789], [222,333,444] ])

with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

print(A)

In [2]:
# What if we want to use a non tensor into a session, use "tf.placeholder" 
# and feed_dict

x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict = {x: "Test World", y : 1123, z : 45.67})
    print (output)




Test World


In [6]:
# Mathematical Operation ->->->->

x = tf.add(5, 2)  # 7
y = tf.sub(10, 4) # 6
z = tf.mul(2, 5)  # 10

with tf.Session() as sess:
    out = sess.run(z)
    print (out)

#x = 10
x = tf.constant(10)

#y = 2
y = tf.constant(2)

z = x/y - 1
z = tf.sub(tf.div(x,y), tf.constant(1))

# TODO: Print z from a session
with tf.Session() as sess:
    output = sess.run(z)
    print (output)    

10
4


In [12]:
# Logistic Classifier -> xW +b
# for the above equation we need a tensor that is modified, use tf.Variable
# the tensor store its states in session, so we must initialize the state of 
# the tensor manually by tf.initialize_all_variables()

x = tf.Variable(5)

#init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    out = sess.run(init)


# Choose weight from a normal distribution prevent one weight from 
# overwhelming another weights, use tf.truncated_normal()

n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

# setting the bias to 0, Here function return a tensor with all zeros

bias = tf.Variable(tf.zeros(n_labels))



In [19]:
# Assign the probablity to each label, which we can then use to classify data.

import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    # TODO: Compute and return softmax(x)
    return np.exp(x) / np.sum(np.exp(x), axis=0)

logits = [3.0, 1.0, 0.2]
print(softmax(logits))


# TensorFlow Softmax

x = tf.nn.softmax([2.0, 1.0, 0.2])
print (x)

[ 0.8360188   0.11314284  0.05083836]
Tensor("Reshape_3:0", shape=(3,), dtype=float32)


In [25]:
# Calculate Cross Entropy After Softmax function

# Reduce Sum
# x = tf.reduce_sum([1, 2, 3, 4, 5])  # 15
# The tf.reduce_sum() function takes an array of numbers and sums them together.

# Natural Log
# x = tf.log(100)  # 4.60517

softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

cross_entropy = - tf.reduce_sum(tf.mul(one_hot, tf.log(softmax)))

with tf.Session() as sess:
    print(sess.run(cross_entropy, feed_dict={softmax: softmax_data, one_hot: one_hot_data}))


0.356675


In [26]:
# Loss = Average cross Entropy ( Minimizing Cross Entropy)
# Measuring Performance
# optimize a Logistic Classification (Design a good loss function to optimize)
# Instead of calculation loss on all over the data, we calculate estimate.
# Adagrad is a modification of SGD which implicitly does the momentum and 
# learning rate decay.


In [33]:
# Batch 
#  batch size will vary so use tf.Variable()
# features = tf.placeholder(tf.float32, [None, n_input])
# labels = tf.placeholder(tf.float32, [None, n_classes])
# None dimension is a placeholder for the batch size. 
# At runtime it can accept any batch size.
# Create Batches

import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]

# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

print(batches(3, example_features, example_labels))

[[[['F11', 'F12', 'F13', 'F14'], ['F21', 'F22', 'F23', 'F24'], ['F31', 'F32', 'F33', 'F34']], [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]], [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]


In [34]:
import math
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np


def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches


learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# TODO: Set batch size
batch_size = 64
assert batch_size is not None, 'You must set the batch size'

init = tf.initialize_all_variables()

with tf.Session() as sess:
	sess.run(init)

	# TODO: Train optimizer on all batches
	# for batch_features, batch_labels in ______
	for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
		sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})

	# Calculate accuracy for test dataset
	test_accuracy = sess.run(
		accuracy,
		feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))


ImportError: No module named 'tensorflow.examples'

In [1]:
# Epochs 
# An epoch is a single forward and backward pass of the whole dataset. This is used to increase the 
# accuracy of the model without requiring more data.

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

batch_size = 128
epochs = 10

train_batches = batches(batch_size, train_features, train_labels)
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch 
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

NameError: name 'batches' is not defined